# {{variable_id}}

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr

from dask.distributed import Client

import catalog
import util

xr.set_options(keep_attrs=True)

## Parameters

In [1]:
casename = 'smyle-fosi.regional.lat-range-basin.000'
component = 'pop'
stream = 'h'

variable_id = 'TEMP'

cluster_scheduler_address = None

In [ ]:
assert component in ['pop']
assert stream in ['h', 'h.ecosys.nday1']

## Connect to cluster

In [ ]:
if cluster_scheduler_address is None:
    cluster, client = util.get_ClusterClient()
    cluster.scale(12)
else:
    client = Client(cluster_scheduler_address)
client

## Load the data

In [ ]:
dsets = catalog.to_dataset_dict(
    case=casename,
    component=component,
    stream=stream,
)
dsets.keys()

## Compute long-term mean and plot

In [ ]:
ds = dsets[f'{casename}.{component}.{stream}']

In [ ]:
dt_wgt = ds.time_bound.diff('d2').squeeze()
dt_wgt /= dt_wgt.sum()
dt_wgt

In [ ]:
ds[variable_id].weighted(dt_wgt).mean('time').isel(z_t=0).plot()